In [ ]:
import sklearn

print(f"scikit-learn version installed in Colab: {sklearn.__version__}")


scikit-learn version installed in Colab: 1.2.2


In [ ]:
import networkx as nx

print(f"NetworkX version installed in Colab: {nx.__version__}")

NetworkX version installed in Colab: 3.3


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.3 MB/s eta 0:00:00


In [3]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score as calculate_f1_score

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load data
edge_index_train = torch.load('/content/drive/MyDrive/PROJECT_90/edge_index_train.pt')
edge_index_test = torch.load('/content/drive/MyDrive/PROJECT_90/edge_index_test.pt')
edge_index_val = torch.load('/content/drive/MyDrive/PROJECT_90/edge_index_val.pt')

features_file_train = '/content/drive/MyDrive/PROJECT_90/feature_matrix_train.txt'
X_train = np.loadtxt(features_file_train)
features_file_test = '/content/drive/MyDrive/PROJECT_90/feature_matrix_test.txt'
X_test = np.loadtxt(features_file_test)
features_file_val = '/content/drive/MyDrive/PROJECT_90/feature_matrix_val.txt'
X_val = np.loadtxt(features_file_val)

labels_test = pd.read_csv("/content/drive/MyDrive/PROJECT_90/test_filtered.csv")
labels_train = pd.read_csv("/content/drive/MyDrive/PROJECT_90/train_filtered.csv")
labels_val = pd.read_csv("/content/drive/MyDrive/PROJECT_90/val_filtered.csv")

y_train = torch.tensor(labels_train['label'].values, dtype=torch.long).to(device)
y_val = torch.tensor(labels_val['label'].values, dtype=torch.long).to(device)
y_true = torch.tensor(labels_test['label'].values, dtype=torch.long).to(device)

# Preprocess features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)

X_test_scaled = scaler.transform(X_test)
X_test = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

X_val_scaled = scaler.transform(X_val)
X_val = torch.tensor(X_val_scaled, dtype=torch.float32).to(device)

class GATNet(torch.nn.Module):
    def __init__(self, in_channels, hidden_dim, out_channels, num_layers):
        super(GATNet, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(GATConv(in_channels, hidden_dim, heads=8))
        for _ in range(num_layers - 1):
            self.convs.append(GATConv(hidden_dim * 8, hidden_dim, heads=8))

        # Output layer
        self.fc = torch.nn.Linear(hidden_dim * 8, out_channels)

    def forward(self, x, edge_index):
        for conv in self.convs:
            x = conv(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc(x)
        return x

# Define model
model = GATNet(in_channels=X_train.shape[1], hidden_dim=32, out_channels=13, num_layers=2).to(device)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Training
def train(model, optimizer, criterion, X_train, edge_train, y_train, X_val, edge_val, y_val, epochs=500, patience=100):
    best_val_loss = float('inf')
    best_val_acc = 0.0
    current_patience = 0
    train_losses = []
    val_losses = []

    train_f1_scores = []  # Initialize list for training F1 scores
    epochss = []

    for epoch in range(epochs):
        epochss.append(epoch)
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train, edge_train)
        loss = criterion(outputs, y_train)
        train_losses.append(loss.cpu().item())
        loss.backward()
        optimizer.step()

        # Compute training accuracy
        _, predicted_train = torch.max(outputs, 1)
        train_acc = torch.sum(predicted_train == y_train).item() / len(y_train)
        # Calculate training F1 score
        train_f1 = calculate_f1_score(y_train.cpu().numpy(), predicted_train.cpu().numpy(), average='weighted')


        # Save training F1 score
        train_f1_scores.append(train_f1)



        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val, edge_val)
            val_loss = criterion(val_outputs, y_val)
            val_losses.append(val_loss)
            # Compute validation accuracy
            _, predicted_val = torch.max(val_outputs, 1)
            val_acc = torch.sum(predicted_val == y_val).item() / len(y_val)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pt')
            current_patience = 0
        else:
            current_patience += 1
            if current_patience >= patience:
                print(f'Early stopping at epoch {epoch}')
                break

        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item()}, Train Acc: {train_acc:.4f},train F1-score:{train_f1:.4f} Val Loss: {val_loss.item()}, Val Acc: {val_acc:.4f}')








# Convert data to appropriate format
edge_train = edge_index_train.to(device)
edge_val = edge_index_val.to(device)
edge_test = edge_index_test.to(device)

# Train the model
train(model, optimizer, criterion, X_train, edge_train, y_train, X_val, edge_val, y_val, epochs=500, patience=100)

# Load the best model
model.load_state_dict(torch.load('best_model.pt'))

# Testing
model.eval()
with torch.no_grad():
    test_outputs = model(X_test, edge_test)
    test_loss = criterion(test_outputs, y_true)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = torch.sum(predicted == y_true).item() / len(y_true)

print(f'Test Loss: {test_loss.item()}, Test Accuracy: {accuracy}')

from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc

# Calculate precision, recall, F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true.cpu().numpy(), predicted.cpu().numpy(), average='weighted')

print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}')





Using device: cpu
Epoch [1/500], Loss: 2.9315733909606934, Train Acc: 0.0665,train F1-score:0.1003 Val Loss: 1.700194001197815, Val Acc: 0.6225
Epoch [2/500], Loss: 1.8956758975982666, Train Acc: 0.6478,train F1-score:0.5121 Val Loss: 1.3951658010482788, Val Acc: 0.6332
Epoch [3/500], Loss: 1.4214621782302856, Train Acc: 0.6363,train F1-score:0.5385 Val Loss: 1.2232803106307983, Val Acc: 0.6506
Epoch [4/500], Loss: 1.176771640777588, Train Acc: 0.6421,train F1-score:0.5689 Val Loss: 1.2200710773468018, Val Acc: 0.5930
Epoch [5/500], Loss: 1.1610640287399292, Train Acc: 0.6466,train F1-score:0.5794 Val Loss: 1.095110535621643, Val Acc: 0.6278
Epoch [6/500], Loss: 1.0101537704467773, Train Acc: 0.6645,train F1-score:0.5941 Val Loss: 0.997617244720459, Val Acc: 0.6586
Epoch [7/500], Loss: 0.989360511302948, Train Acc: 0.6642,train F1-score:0.6190 Val Loss: 1.022924542427063, Val Acc: 0.6412
Epoch [8/500], Loss: 0.9605795741081238, Train Acc: 0.6794,train F1-score:0.6113 Val Loss: 0.977183

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Load the best GIN model
model.load_state_dict(torch.load('best_model.pt'))

# Get the output of the GIN model for the training, validation, and test sets
model.eval()
with torch.no_grad():
    train_outputs = model(X_train, edge_train).cpu().numpy()
    val_outputs = model(X_val, edge_val).cpu().numpy()
    test_outputs = model(X_test, edge_test).cpu().numpy()

# Train a decision tree classifier
decision_tree = DecisionTreeClassifier(max_depth=1)
decision_tree.fit(train_outputs, y_train.cpu().numpy())

# Predict labels using the decision tree
train_pred = decision_tree.predict(train_outputs)
val_pred = decision_tree.predict(val_outputs)
test_pred = decision_tree.predict(test_outputs)

# Evaluate decision tree performance
train_acc = accuracy_score(y_train.cpu().numpy(), train_pred)
val_acc = accuracy_score(y_val.cpu().numpy(), val_pred)
test_acc = accuracy_score(y_true.cpu().numpy(), test_pred)

print(f'Training Accuracy (Decision Tree): {train_acc}')
print(f'Validation Accuracy (Decision Tree): {val_acc}')
print(f'Test Accuracy (Decision Tree): {test_acc}')


Training Accuracy (Decision Tree): 0.7121697059918124
Validation Accuracy (Decision Tree): 0.6907630522088354
Test Accuracy (Decision Tree): 0.6796246648793566


In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Load the best GIN model
model.load_state_dict(torch.load('best_model.pt'))

# Get the output of the GIN model for the training, validation, and test sets
model.eval()
with torch.no_grad():
    train_outputs = model(X_train, edge_train).cpu().numpy()
    val_outputs = model(X_val, edge_val).cpu().numpy()
    test_outputs = model(X_test, edge_test).cpu().numpy()

# Concatenate the GIN model output with the original feature matrices
X_train_combined = np.concatenate((X_train.cpu().numpy(), train_outputs), axis=1)
X_val_combined = np.concatenate((X_val.cpu().numpy(), val_outputs), axis=1)
X_test_combined = np.concatenate((X_test.cpu().numpy(), test_outputs), axis=1)

# Train a decision tree classifier
decision_tree = DecisionTreeClassifier(max_depth=2)
decision_tree.fit(X_train_combined, y_train.cpu().numpy())

# Predict labels using the decision tree
train_pred = decision_tree.predict(X_train_combined)
val_pred = decision_tree.predict(X_val_combined)
test_pred = decision_tree.predict(X_test_combined)

# Evaluate decision tree performance
train_acc = accuracy_score(y_train.cpu().numpy(), train_pred)
val_acc = accuracy_score(y_val.cpu().numpy(), val_pred)
test_acc = accuracy_score(y_true.cpu().numpy(), test_pred)

print(f'Training Accuracy (Decision Tree): {train_acc}')
print(f'Validation Accuracy (Decision Tree): {val_acc}')
print(f'Test Accuracy (Decision Tree): {test_acc}')


Training Accuracy (Decision Tree): 0.7733531819873465
Validation Accuracy (Decision Tree): 0.7295850066934404
Test Accuracy (Decision Tree): 0.7144772117962467


In [8]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Load the best GIN model
model.load_state_dict(torch.load('best_model.pt'))

# Get the output of the GIN model for the training, validation, and test sets
model.eval()
with torch.no_grad():
    train_outputs = model(X_train, edge_train).cpu().numpy()
    val_outputs = model(X_val, edge_val).cpu().numpy()
    test_outputs = model(X_test, edge_test).cpu().numpy()

# Train an SVM classifier
svm_classifier = SVC(kernel='linear')  # You can specify different kernel functions (e.g., 'linear', 'poly', 'rbf', etc.)
svm_classifier.fit(train_outputs, y_train.cpu().numpy())

# Predict labels using the SVM classifier
train_pred = svm_classifier.predict(train_outputs)
val_pred = svm_classifier.predict(val_outputs)
test_pred = svm_classifier.predict(test_outputs)

# Evaluate SVM performance
train_acc = accuracy_score(y_train.cpu().numpy(), train_pred)
val_acc = accuracy_score(y_val.cpu().numpy(), val_pred)
test_acc = accuracy_score(y_true.cpu().numpy(), test_pred)

print(f'Training Accuracy (SVM): {train_acc}')
print(f'Validation Accuracy (SVM): {val_acc}')
print(f'Test Accuracy (SVM): {test_acc}')


Training Accuracy (SVM): 0.9560848529959062
Validation Accuracy (SVM): 0.9089692101740294
Test Accuracy (SVM): 0.8699731903485255


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Load the best GIN model
model.load_state_dict(torch.load('best_model1.pt'))

# Get the output of the GIN model for the training, validation, and test sets
model.eval()
with torch.no_grad():
    train_outputs = model(X_train, edge_train).cpu().numpy()
    val_outputs = model(X_val, edge_val).cpu().numpy()
    test_outputs = model(X_test, edge_test).cpu().numpy()

# Combine the output of the GIN model with the original features
X_train_combined = np.concatenate((X_train.cpu().numpy(), train_outputs), axis=1)
X_val_combined = np.concatenate((X_val.cpu().numpy(), val_outputs), axis=1)
X_test_combined = np.concatenate((X_test.cpu().numpy(), test_outputs), axis=1)

# Train an SVM classifier
svm_classifier = SVC(kernel='linear')  # You can specify different kernel functions (e.g., 'linear', 'poly', 'rbf', etc.)
svm_classifier.fit(X_train_combined, y_train.cpu().numpy())

# Predict labels using the SVM classifier
train_pred = svm_classifier.predict(X_train_combined)
val_pred = svm_classifier.predict(X_val_combined)
test_pred = svm_classifier.predict(X_test_combined)

# Evaluate SVM performance
train_acc = accuracy_score(y_train.cpu().numpy(), train_pred)
val_acc = accuracy_score(y_val.cpu().numpy(), val_pred)
test_acc = accuracy_score(y_true.cpu().numpy(), test_pred)

print(f'Training Accuracy (SVM): {train_acc}')
print(f'Validation Accuracy (SVM): {val_acc}')
print(f'Test Accuracy (SVM): {test_acc}')
